# 1.0 Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import RepeatedKFold, RandomizedSearchCV
from sklearn.metrics import r2_score, max_error, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error


# 2.0 Leitura do dataset

In [2]:
df = pd.read_csv("data/dataset.csv")

# 3.0 Pré-processamento

In [3]:
# Retirar linhas com valores vazios (Em 2013 só tem pontos do enduro e totais)
df = df.dropna().reset_index(drop=True)

In [4]:
# Pontuação da prova de segurança englobava outras provas até 2014
df = df.drop(df[df['Ano'] < 2014].index).reset_index(drop=True)

In [5]:
# Otimizando a alocação de espaço das colunas numéricas
df["Ano"] = pd.to_numeric(df["Ano"], downcast='integer')
df["Numero"] = pd.to_numeric(df["Numero"], downcast='integer')
df["Numero"] = pd.to_numeric(df["Numero"], downcast='float')
df["Seguranca"] = pd.to_numeric(df["Seguranca"], downcast='float')
df["Dinamicas"] = pd.to_numeric(df["Dinamicas"], downcast='float')
df["Enduro"] = pd.to_numeric(df["Enduro"], downcast='float')
df["Total"] = pd.to_numeric(df["Total"], downcast='float')

# 4.0 Criação do modelo de Machine Learning

## 4.1 Definição das variáveis do modelo

In [6]:
variaveis_analisadas = ['Seguranca', 'Projeto', 'Dinamicas', 'Enduro']
x = df[variaveis_analisadas]
y = df['Posicao']

## 4.2 Normalização dos inputs

In [7]:
mms = MinMaxScaler()
for variavel in variaveis_analisadas:
    x[variavel] = mms.fit_transform(x[variavel].values.reshape((-1, 1)))
    pickle.dump(mms, open("persistence/mms_{}.pkl".format(variavel), 'wb'))

C:\Users\mmath\AppData\Local\Temp\ipykernel_17432\4091827902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[variavel] = mms.fit_transform(x[variavel].values.reshape((-1, 1)))
C:\Users\mmath\AppData\Local\Temp\ipykernel_17432\4091827902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[variavel] = mms.fit_transform(x[variavel].values.reshape((-1, 1)))
C:\Users\mmath\AppData\Local\Temp\ipykernel_17432\4091827902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

## 4.3 Separação do dataset em conjuntos de treino e teste

In [8]:
random_state = 1
xtrain, xtest, ytrain, ytest = train_test_split(x.values, y.values, test_size=0.1, random_state=random_state)

## 4.4 Tuning dos hiper-parâmetros

In [9]:
# Definindo o modelo regressivo e o range de hiper-parâmetros viáveis
model = SVR()
params = {'kernel': ['linear', 'rbf', 'sigmoid'],
          'gamma': [0.00001 * pow(10, x) for x in range(10)],
          'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 20000, 30000, 40000, 50000, 100000],
          'epsilon': [0.01 * pow(10, x) for x in range(3)]}

In [10]:
# Definindo a técnica de validação cruzada a ser utilizada. Nesse caso será utilizada a Repeated K-Fold Cross-Validation.
cv = RepeatedKFold(n_splits=5,
                   n_repeats=1,
                   random_state=random_state)

# Definindo o tipo de busca que será realizada. Nesse caso será utilizada uma busca randômica pelas combinações de hiper-parâmetros.
search = RandomizedSearchCV(estimator=model,
                            param_distributions=params,
                            cv=cv,
                            n_iter=100,
                            random_state=random_state,
                            scoring='max_error')

# Busca dos hiper-parâmetros a partir do grupo de treinamento.
search.fit(xtrain, ytrain)

# Atualizando o modelo com a melhor combinação de hiper-parâmetros encontrada.
model = search.best_estimator_

## 4.5 Treinamento final do modelo de Machine Learning

In [11]:
model.fit(xtrain, ytrain)
pickle.dump(model, open("persistence/model.pkl", 'wb'))

## 4.6 Predição do grupo de teste

In [12]:
ypred = np.ceil(model.predict(xtest))

## 4.7 Avaliação do modelo de ML

In [13]:
METRICS = {
    "r2_score": lambda true, pred: r2_score(true, pred),
    "max_error": lambda true, pred: max_error(true, pred),
    "mean_absolute_error": lambda true, pred: mean_absolute_error(true, pred),
    "root_mean_squared_error": lambda true, pred: mean_squared_error(true, pred) ** 0.5,
    "mean_absolute_percentage_error": lambda true, pred: mean_absolute_percentage_error(true, pred)
}
for key in METRICS.keys():
    print("{:33}: {:5.2f}".format(key, METRICS[key](ytest, ypred)))

r2_score                         :  0.95
max_error                        : 11.00
mean_absolute_error              :  3.68
root_mean_squared_error          :  4.62
mean_absolute_percentage_error   :  0.15
